In [0]:
!pip install shap
!pip install importlib
!pip install lifelines
!pip install plotly

In [0]:
from google.colab import auth
from google.colab import drive
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')

In [3]:
!git clone https://github.com/gowun/MedicalDataScience.git

Cloning into 'MedicalDataScience'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 241 (delta 111), reused 217 (delta 87), pack-reused 0
Receiving objects: 100% (241/241), 940.36 KiB | 20.90 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [4]:
from MedicalDataScience.modeling_tools import  utils as ut
from MedicalDataScience.modeling_tools import clustering as cl
from MedicalDataScience.modeling_tools import preprocessing as pp
from MedicalDataScience.modeling_tools import tree_modeling as tm
from MedicalDataScience.modeling_tools import linear_modeling as lm
from MedicalDataScience.modeling_tools import survival_analysis as sa
from MedicalDataScience.modeling_tools import metrics as mt
from MedicalDataScience.modeling_tools import figure_eda as fe

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [5]:
home_path = '/content/gdrive/My Drive/'
df = ut.load_data(home_path + 'bladdercancer/TABLE용_191018.csv', 'csv')
duration_cols = list(filter(lambda x: 'OS' in x or 'PFS' in x, df.columns))
df.columns = list(map(lambda x: x.split('\n')[0] + ' ' + x.split('\n')[-1] if '\n' in x else x, df.columns))
df.columns = list(map(lambda x: 'Prostate involvement' if 'a_Prostate involvement' in x else x, df.columns))
df.columns = list(map(lambda x: x.strip(' '), df.columns))
X_cols = sorted(set(filter(lambda x: x.startswith('a_') and 'variant' not in x and 'Percent' not in x and 'Intensity' not in x and 'cho' not in x, df.columns)) 
- {'a_chemo period(M)', 'a_Neoadjuvant Regimen', 'a_Death', 'a_cause of death', 'a_Cycles', 'a_T_minimum', 'a_Proper muscle', 'a_Relapse PD', 'a_Solid(%)', 'a_organ', 'a_LVI', 'a_Stage', 'a_procedure'})
X_cols = X_cols + list(filter(lambda x: 'Percent.3' in x, df.columns)) + ['sum_variant'] 
cat_cols = ['a_diagnosis']
org_X = df[X_cols]
org_X.columns = list(map(lambda x: x[2:] if x.startswith('a_') else x, org_X.columns))
org_X.columns = list(map(lambda x: x[:-2] if x.endswith('.3') else x, org_X.columns))
org_X.columns = list(map(lambda x: x[0].upper() + x[1:], org_X.columns))
org_X.columns

Index(['1973 WHO grade', 'CK14 H-score', 'CK14 IRS scoring system',
       'CK20 H-score', 'CK20 IRS scoring system', 'CK5/6 H-score',
       'CK5/6 IRS scoring system', 'FOXA1 H-score', 'FOXA1 IRS scoring system',
       'GATA3 H-score', 'GATA3 IRS scoring system', 'Gender', 'Grade', 'Age',
       'Diagnosis', 'CK5/6 Percent', 'CK14 Percent', 'CK20 Percent',
       'GATA3 Percent', 'FOXA1 Percent', 'Sum_variant'],
      dtype='object')

In [6]:
variant_types = ['Glandular', 'Lymphoma-like', 'Sarcomatoid', 'Micropapillary', 'Giant', 'Squamous', 'Nested', 'Others']
variants = pp.preprocessing_category(df[['a_variant 1 name', 'a_variant 2 name', 'a_variant 3 name']])
rm_cols = list(filter(lambda x: x.endswith('0.0'), variants.columns))
variants = variants.drop(rm_cols, 1)
total_variants = pd.DataFrame()
remain_cols = set(variants.columns)
for i in range(1, 8):
  ss = str(i) + '.0'
  m_cols = list(filter(lambda x: x.endswith(ss), variants.columns))
  total_variants[variant_types[i-1]] = variants[m_cols].sum(1)
  remain_cols -= set(m_cols)
total_variants[variant_types[-1]] = variants[list(remain_cols)].sum(1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.



In [8]:
num_cols = sorted(set(org_X.columns) - {'Diagnosis'})
X_num = pp.preprocessing_numeric(org_X[num_cols], [999, '999'])
X_cat = pp.preprocessing_category(org_X[['Diagnosis']], given_colnames=['IUC', 'IUC_InSitu'])

/content/MedicalDataScience/modeling_tools/preprocessing.py:146: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/content/MedicalDataScience/modeling_tools/preprocessing.py:147: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/sklearn/pr

In [0]:
X = pd.concat([X_num, X_cat, total_variants], 1)
y = df['b_pathologic response']
event = df['a_Death'].values
durations = df[duration_cols[:3]]
ut.save_as_file_colab([X, y, event, durations], home_path + 'bladdercancer/X_y_event_durations.pkl', 'pickle')

In [12]:
X.columns

Index(['1973 WHO grade', 'Age', 'CK14 H-score', 'CK14 IRS scoring system',
       'CK14 Percent', 'CK20 H-score', 'CK20 IRS scoring system',
       'CK20 Percent', 'CK5/6 H-score', 'CK5/6 IRS scoring system',
       'CK5/6 Percent', 'FOXA1 H-score', 'FOXA1 IRS scoring system',
       'FOXA1 Percent', 'GATA3 H-score', 'GATA3 IRS scoring system',
       'GATA3 Percent', 'Gender', 'Grade', 'Sum_variant', 'IUC', 'IUC_InSitu',
       'Glandular', 'Lymphoma-like', 'Sarcomatoid', 'Micropapillary', 'Giant',
       'Squamous', 'Nested', 'Others'],
      dtype='object')

In [13]:
event

array([0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0])

In [14]:
durations

,OS_initial diagnosis(M),OS_chemo start,PFS_surgery
0,35,34,31
1,17,16,8
2,38,38,35
3,19,19,16
4,18,17,15
...,...,...,...
86,12,11,9
87,51,50,47
88,62,60,56
89,59,59,55
